<h1> 2021-2 캡스톤 디자인 </h1>  


<h3>목표 : 대출량 예측 및 우선 대출 도서 선정 모델 개발</h3>  

데이터 수집 :  
1.도서관 정보나루 API 에서 국립세종도서관의 대출 데이터 수집  
2.국립중앙도서관 Open API에서 서지정보 수집 

In [1]:
import pandas as pd
import numpy as np
import json
from urllib.request import urlopen, Request
import urllib
import requests
import re
from konlpy.tag import Okt
from gensim import models
import time
from tqdm import trange
from PIL import Image
import joblib
import tensorflow as tf
from collections import Counter
from hyperopt import hp, fmin, tpe, Trials, STATUS_OK
from sklearn.model_selection import train_test_split 
import xgboost as xgb

C:\Users\soymi\anaconda3\lib\site-packages\gensim\similarities\__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [12]:
jn_key = 'f8a1ee19fb60a1fe7d89fbefd5a1ee729d4a67be99b748cdef74e3b1381f9e6e'

In [36]:
isbn = []
loan_time = []


url = f'http://data4library.kr/api/itemSrch?&libCode=50001&authKey={jn_key}&pageNo={1}&pageSize=100&format=json&startDt=2020-08-31&endDt=2021-08-31'
res = urlopen(url)
res_m = res.read().decode('utf-8')
res_json = json.loads(res_m)
pc = res_json['response']['numFound'] // 100 if (res_json['response']['numFound'] % 100) == 0 else (res_json['response']['numFound'] // 100) + 1

for i in trange(1, pc + 1):
    url = f'http://data4library.kr/api/itemSrch?&libCode=50001&authKey={jn_key}&pageNo={i}&pageSize=100&format=json&startDt=2020-08-31&endDt=2021-08-31'
    res = urlopen(url)
    res_m = res.read().decode('utf-8')
    res_json = json.loads(res_m)
    for j in res_json['response']['docs']:
        isbn.append(j['doc']['isbn13'])
        loan_time.append(j['doc']['reg_date'])

100%|████████████████████████████████████████████████████████████████████████████████| 851/851 [26:38<00:00,  1.88s/it]


In [38]:
loan = pd.DataFrame({'isbn' : isbn, 'loan_time' : loan_time})
loan.to_csv('loan.csv',index = False)
loan

,isbn,loan_time
0,9791160947427,2021-08-31
1,9791191825022,2021-08-31
2,9788934989349,2021-08-31
3,9788998940010,2021-08-31
4,9791187512578,2021-08-31
...,...,...
85047,9791190582308,2020-08-31
85048,9791158511807,2020-08-31
85049,9791196572846,2020-08-31
85050,9791190242455,2020-08-31


In [13]:
oa_key = '578ca4ba507631e4a9b621f4029400eac427aaf6071b45611e599387b637b6dc'

In [68]:
author = []
publish_date = []
title = []
title_image_url = []
pages = []
price = []

reader = []
pub_type = []
subject_1 = []
subject_2 = []

isbn = loan.isbn.value_counts().index
blank_isbn = []
isbn_loop = []
for isb in trange(len(isbn)):
    i = isbn[isb] 
    url = f'http://seoji.nl.go.kr/landingPage/SearchApi.do?cert_key={oa_key}&isbn={i}&result_style=json&page_size=1&page_no=1'
    res = requests.get(url)
    try:
        info = res.json()['docs'][0]
    except:
        blank_isbn.append(i)
        continue
        
    isbn_loop.append(i)
    title.append(info['TITLE'])
    title_image_url.append(info['TITLE_URL'])
    
    if info['REAL_PUBLISH_DATE'] == '':
        publish_date.append(info['PUBLISH_PREDATE'])
    else:
        publish_date.append(info['REAL_PUBLISH_DATE'])
        
    if info['REAL_PRICE'] == '':
        price.append(info['PRE_PRICE'])
    else:
        price.append(info['REAL_PRICE'])
        
    try:
        ea_code = info['EA_ADD_CODE']
        subject_1.append(info['EA_ADD_CODE'][2])
        subject_2.append(info['EA_ADD_CODE'][3])
        reader.append(info['EA_ADD_CODE'][0])
        pub_type.append(info['EA_ADD_CODE'][1])
    except:
        subject_1.append(np.NaN)
        subject_2.append(np.NaN)
        reader.append(np.NaN)
        pub_type.append(np.NaN)
    pages.append(info['PAGE'])
    author.append(info['AUTHOR'])

100%|██████████████████████████████████████████████████████████████████████████| 42279/42279 [1:00:54<00:00, 11.57it/s]


In [ ]:
book = pd.DataFrame({'isbn' : isbn_loop, 'author' : author, 'pub_date' : publish_date, 'title' : title, 'title_image_url' : title_image_url, 'pages' : pages, 'price' :  price, 'reader' : reader, 'pub_type' : pub_type, 'subject_1' : subject_1, 'subject_2' : subject_2})
book.to_csv('book.csv', index = False)

In [14]:
book = pd.read_csv('book.csv')
book

,isbn,author,pub_date,title,title_image_url,pages,price,reader,pub_type,subject_1,subject_2
0,9788984374164,지은이: 기욤 뮈소 ;옮긴이: 양영란,20201124.0,인생은 소설이다,https://image.aladin.co.kr/product/25594/44/co...,311 p.,15000,0.0,3.0,8.0,6.0
1,9791160953268,충청남도교육청교육연수원 [편],20190401.0,(2019) 제16기 핵심인재 양성과정,NaN,"xvi, 467 p.",비매품/무료,1.0,3.0,3.0,5.0
2,9788934986652,글:오은영 ;그림: 차상미,20201025.0,어떻게 말해줘야 할까,http://seoji.nl.go.kr/fu/ecip/dbfiles/CIP_FILE...,397 p.,17500,1.0,3.0,5.0,9.0
3,9791190808125,라곰씨 글 ;차차 그림,20210129.0,(무엇이든 해결단) 허팝 연구소,https://image.aladin.co.kr/product/26240/52/co...,124 p.,13000,7.0,4.0,8.0,1.0
4,9791164137176,흔한남매 원작 ;김언정 글 ;차차 그림,20201215.0,(흔한남매) 별난 방탈출,https://image.aladin.co.kr/product/25813/3/cov...,126 p.,12000,7.0,7.0,8.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...
37312,9788954677547,바버라 오클리 지음 ;김아림 옮김,20210302.0,이과형 두뇌 활용법,https://bookthumb-phinf.pstatic.net/cover/180/...,369 p.,17000,0.0,3.0,4.0,0.0
37313,9791189329068,권미정;김은주;백난희;양승옥;유율희;윤미옥;이지형;임희,20200310.0,당신의 행복은 안녕하신가요?,https://image.aladin.co.kr/product/24131/1/cov...,307 p.,20000,0.0,3.0,1.0,9.0
37314,9788997728404,글: 마크 서머셋 ; 그림: 로완 서머셋,20131126.0,똑똑해지는 약,http://seoji.nl.go.kr/fu/ecip/dbfiles/CIP_FILE...,NaN,15000,7.0,7.0,8.0,9.0
37315,9791189773175,저자 : 윤혜정,20200601.0,"민들레야, 잘가",https://image.aladin.co.kr/product/24265/18/co...,42,"15,000",0.0,7.0,8.0,0.0


In [15]:
book.title_image_url.value_counts()

https://image.aladin.co.kr/product/25836/43/cover/8949105292_1.jpg                  2
http://seoji.nl.go.kr/fu/ecip/dbfiles/CIP_FILES_TBL/2020/07/08/9791196541279.jpg    1
https://image.aladin.co.kr/product/24615/74/cover/k712631272_1.jpg                  1
http://seoji.nl.go.kr/fu/ecip/dbfiles/CIP_FILES_TBL/2020/12/09/9791190971171.jpg    1
https://image.aladin.co.kr/product/24616/12/cover/k202631273_1.jpg                  1
                                                                                   ..
http://seoji.nl.go.kr/fu/ecip/dbfiles/CIP_FILES_TBL/2020/07/01/9788967451028.JPG    1
https://image.aladin.co.kr/product/25694/16/cover/k792736569_1.jpg                  1
http://seoji.nl.go.kr/fu/ecip/dbfiles/CIP_FILES_TBL/2020/08/02/9791190898089.jpg    1
http://image.aladin.co.kr/product/10336/89/cover/k082536589_1.jpg                   1
https://image.aladin.co.kr/product/25507/0/cover/8946069732_1.jpg                   1
Name: title_image_url, Length: 30135, dtype: int64

In [16]:
#국중 open API에 없던 것들을 도서관정보나루 API에서 받아옴

no_image = book.loc[(book.title_image_url.isna()) | (book.title_image_url == ''), ]
isbn = no_image.isbn
title_image_url = []

for isb in trange(len(isbn)):
    try:
        i = isbn.iloc[isb]
        url = f'http://data4library.kr/api/srchDtlList?authKey={jn_key}&isbn13={i}&format=json'
        res = requests.get(url)
        title_image_url.append(res.json()['response']['detail'][0]['book']['bookImageURL'])
    except:
        title_image_url.append(np.NaN)

100%|██████████████████████████████████████████████████████████████████████████████| 7181/7181 [38:52<00:00,  3.08it/s]


In [17]:
for isb in trange(len(isbn)):
    i = isbn.iloc[isb]
    book.loc[book.isbn == i,'title_image_url'] = title_image_url[isb]

100%|████████████████████████████████████████████████████████████████████████████| 7181/7181 [00:06<00:00, 1145.24it/s]


In [18]:
book.to_csv('book.csv', index = False)

In [2]:
book = pd.read_csv('book.csv', dtype = {'isbn' : 'object'})

In [3]:
only_image = (book.loc[pd.DataFrame({'col1' : book.title_image_url.notna(),'col2' : book.title_image_url != ''}).all(axis = 1), ['isbn','title_image_url']]).reset_index(drop=True)
image_isbn = []

for i in trange(len(only_image)):
    try:
        urllib.request.urlretrieve(only_image.iloc[i,1], f'C:/Users/soymi/Desktop/lib_git/2021-2Capstonedesign/image/{only_image.iloc[i,0]}.jpg')
        image_isbn.append(only_image.iloc[i,0])
    except:
        continue

100%|██████████████████████████████████████████████████████████████████████████| 30136/30136 [1:28:01<00:00,  5.71it/s]


In [4]:
joblib.dump(image_isbn, 'image_isbn_list.pkl')

['image_isbn_list.pkl']

<h2>데이터 수집 끝 - 전처리시작</h2>

<h3>1차적으로 이미지가 주제 혹은 특정한 의미를 나타낼정도로 의미를 갖는지 보기 위해 이미지로 주제번호를 예측하는 모델, 표제 등장 단어로 주제번호를 예측하는 모델을 만들어봄</h3>

In [8]:
book = pd.read_csv('book.csv', dtype = {'isbn' : 'object'})
image_isbn = joblib.load('image_isbn_list.pkl')

In [9]:
len(image_isbn)

30123

In [10]:
def get_sbj(X):
    return book.loc[book.isbn == X, 'subject_1'].item()

image_subject = list(map(get_sbj, image_isbn))

In [11]:
del(book)
image_df = pd.DataFrame({'isbn' : image_isbn, 'subject' : image_subject})
image_df = image_df.loc[image_df.subject.notna(),:]

In [12]:
image_list = []
image_isbn = []
for isb in trange(len(image_df.isbn)):
    try:
        isbn = image_df.isbn.iloc[isb]
        img = tf.keras.preprocessing.image.load_img(f'C:/Users/soymi/Desktop/lib_git/2021-2Capstonedesign/image/{isbn}.jpg', target_size = (224, 224))
        img_a = np.expand_dims(tf.keras.preprocessing.image.img_to_array(img), axis = 0)
        img_a = tf.keras.applications.efficientnet.preprocess_input(img_a)
        image_list.append(img_a)
        image_isbn.append(isbn)
    except:
        continue
        
image_list = np.concatenate(image_list)

image_df = image_df.loc[image_df.isbn.isin(image_isbn),:]

 27%|████████████████████▊                                                       | 8224/30055 [01:41<03:36, 100.61it/s]C:\Users\soymi\anaconda3\lib\site-packages\PIL\Image.py:2855: DecompressionBombWarning: Image size (95448150 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(
 42%|███████████████████████████████▌                                            | 12480/30055 [02:36<03:37, 80.95it/s]C:\Users\soymi\anaconda3\lib\site-packages\PIL\Image.py:2855: DecompressionBombWarning: Image size (126802270 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(
 51%|█████████████████████████████████████▉                                     | 15215/30055 [03:13<01:55, 128.73it/s]C:\Users\soymi\anaconda3\lib\site-packages\PIL\Image.py:2855: DecompressionBombWarning: Image size (157754760 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(
 56%|████████████

In [13]:
label = image_df.subject.astype('int8')
label

0        8
1        5
2        8
3        8
4        8
        ..
30118    4
30119    1
30120    8
30121    8
30122    8
Name: subject, Length: 30037, dtype: int8

In [14]:
image_list.shape

(30037, 224, 224, 3)

In [15]:
joblib.dump(label, 'model1_label.pkl')
joblib.dump(image_list, 'model1_X.pkl')

['model1_X.pkl']

In [2]:
label = joblib.load('model1_label.pkl')
image_list = joblib.load('model1_X.pkl')

In [3]:
X_train, X_val, y_train, y_val = train_test_split(image_list, label, shuffle=True, stratify = label, train_size = .8, random_state = 42)
del(image_list)

In [4]:
label.value_counts()

8    11996
3     6135
5     2100
9     2030
1     1951
2     1472
6     1430
4     1330
0     1167
7      426
Name: subject, dtype: int64

In [5]:
X_train.shape

(24029, 224, 224, 3)

In [14]:
tf.random.set_seed(42)
effnetb4 = tf.keras.applications.EfficientNetB4(weights = 'imagenet', include_top = False, input_shape = (224,224,3))
for layer in effnetb4.layers:
    layer.trainable = False

fcmlp = effnetb4.output
fcmlp = tf.keras.layers.GlobalAveragePooling2D()(fcmlp)
fcmlp = tf.keras.layers.Dense(1024, activation = 'selu', kernel_initializer = tf.keras.initializers.he_normal(seed = 42))(fcmlp)
fcmlp = tf.keras.layers.BatchNormalization()(fcmlp)
fcmlp = tf.keras.layers.Dense(512, activation = 'selu', kernel_initializer = tf.keras.initializers.he_normal(seed = 42))(fcmlp)
fcmlp = tf.keras.layers.BatchNormalization()(fcmlp)
fcmlp = tf.keras.layers.Dense(256, activation = 'selu', kernel_initializer = tf.keras.initializers.he_normal(seed = 42))(fcmlp)
fcmlp = tf.keras.layers.BatchNormalization()(fcmlp)
fcmlp = tf.keras.layers.Dense(10, activation = 'softmax')(fcmlp)

model = tf.keras.Model(inputs = effnetb4.input, outputs = fcmlp)

optimizer = tf.keras.optimizers.Nadam(learning_rate = 0.01, beta_1=0.95, beta_2=0.99, epsilon = 1e-07)
model.compile(loss = 'sparse_categorical_crossentropy', optimizer = optimizer, metrics = ['accuracy'])

In [16]:
callback = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=5)
history = model.fit(X_train, y_train, epochs = 30, batch_size = 64, callbacks = [callback], validation_data=(X_val, y_val))

Epoch 1/30
376/376 [==============================] - 60s 160ms/step - loss: 1.6577 - accuracy: 0.4820 - val_loss: 1.6865 - val_accuracy: 0.4762
Epoch 2/30
376/376 [==============================] - 59s 156ms/step - loss: 1.5716 - accuracy: 0.5051 - val_loss: 1.6277 - val_accuracy: 0.5060
Epoch 3/30
376/376 [==============================] - 59s 156ms/step - loss: 1.5042 - accuracy: 0.5207 - val_loss: 1.5706 - val_accuracy: 0.5148
Epoch 4/30
376/376 [==============================] - 59s 157ms/step - loss: 1.4213 - accuracy: 0.5430 - val_loss: 1.5960 - val_accuracy: 0.5082
Epoch 5/30
376/376 [==============================] - 59s 157ms/step - loss: 1.3278 - accuracy: 0.5703 - val_loss: 1.6540 - val_accuracy: 0.5003
Epoch 6/30
376/376 [==============================] - 59s 157ms/step - loss: 1.2351 - accuracy: 0.5962 - val_loss: 1.6694 - val_accuracy: 0.4942
Epoch 7/30
376/376 [==============================] - 59s 156ms/step - loss: 1.1455 - accuracy: 0.6237 - val_loss: 1.6796 - val_ac

In [19]:
for layer in effnetb4.layers[324:]:
    layer.trainable = True

callback = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=5)
history2 = model.fit(X_train, y_train, epochs = 50, batch_size = 64, callbacks = [callback], validation_data=(X_val, y_val))

Epoch 1/50
376/376 [==============================] - 59s 156ms/step - loss: 0.7893 - accuracy: 0.7347 - val_loss: 2.0868 - val_accuracy: 0.4482
Epoch 2/50
376/376 [==============================] - 59s 156ms/step - loss: 0.7152 - accuracy: 0.7540 - val_loss: 2.1401 - val_accuracy: 0.4915
Epoch 3/50
376/376 [==============================] - 59s 158ms/step - loss: 0.7026 - accuracy: 0.7603 - val_loss: 2.0918 - val_accuracy: 0.4958
Epoch 4/50
376/376 [==============================] - 59s 158ms/step - loss: 0.6681 - accuracy: 0.7715 - val_loss: 2.2057 - val_accuracy: 0.4587
Epoch 5/50
376/376 [==============================] - 59s 157ms/step - loss: 0.6185 - accuracy: 0.7901 - val_loss: 2.3274 - val_accuracy: 0.4562
Epoch 6/50
376/376 [==============================] - 59s 157ms/step - loss: 0.5810 - accuracy: 0.8000 - val_loss: 2.3029 - val_accuracy: 0.4489
Epoch 7/50
376/376 [==============================] - 59s 157ms/step - loss: 0.5439 - accuracy: 0.8126 - val_loss: 2.4132 - val_ac

특정 sample이 train에 가있는가 validation에 가있는가에 따라 차이가 굉장히 큼 -> 이거 어떻게할지 고민해봐야할듯? 

일단 train set에서 accuracy가 .8이 찍히는것을 보면 표지와 주제는 충분히 상관성이 있어보임. 자세한건 교수님과 상담 

<h3> 그 다음은 표제에서 사용된 단어로 주제를 표현할 수 있을지 알아볼 것임 </h3>